In [1]:
!pip install --upgrade pip

In [2]:
!pip install --no-cache-dir datasets

In [3]:
!pip install --upgrade pip
!pip install --no-cache-dir accelerate

In [4]:
!pip install --no-cache-dir transformers

In [5]:
!pip install --upgrade transformers

In [6]:
import sys
print(sys.executable)


/usr/bin/python


In [8]:
!pip install accelerate

In [12]:

import os
import json
import torch
from transformers import AutoProcessor, Gemma3ForConditionalGeneration

def main():
    model_id = "google/gemma-3-4b-it"
    print(f"Loading model: {model_id}")

    model = Gemma3ForConditionalGeneration.from_pretrained(
        model_id,
        trust_remote_code=True,
        use_auth_token=True,  # HF Token が必要な場合
        device_map="auto"
    ).eval()

    processor = AutoProcessor.from_pretrained(model_id, trust_remote_code=True, use_auth_token=True)

    input_file = "questions_ja.jsonl"
    output_file = "gemma3_output.jsonl"

    with open(input_file, "r", encoding="utf-8") as f_in, \
         open(output_file, "w", encoding="utf-8") as f_out:

        for line in f_in:
            line = line.strip()
            if not line:
                continue

            item = json.loads(line)
            question_id = item["question_id"]
            image_path = item["image"]          
            category = item["category"]
            image_cat = item["image_category"]
            context = item["context"]
            question_text = item["text"]

            # 画像をディレクトリ直下に入れる
            # 存在確認をしたい場合は次のようにもできる:
            #   rel_path = os.path.join(".", image_path)
            #   if not os.path.exists(rel_path):
            #       print(f"Image not found: {rel_path}")
            #       continue
            #
            # ここでは単純に相対パスで指定している
            image_source = image_path  

            messages = [
                {
                    "role": "system",
                    "content": [
                        {"type": "text", "text": "You are a helpful Japanese vision-language assistant."}
                    ]
                },
                {
                    "role": "user",
                    "content": [
                        # ここで "type": "image" には file:// は付けずに、ファイルパスをそのまま指定する
                        {"type": "image", "image": image_source},
                        {"type": "text", "text": f"{context}\n\n{question_text}"}
                    ]
                }
            ]

            inputs = processor.apply_chat_template(
                messages,
                add_generation_prompt=True,
                tokenize=True,
                return_dict=True,
                return_tensors="pt"
            ).to(model.device)

            input_len = inputs["input_ids"].shape[-1]

            with torch.inference_mode():
                generated = model.generate(
                    **inputs,
                    max_new_tokens=128,
                    do_sample=False
                )

            gen_part = generated[0][input_len:]
            decoded_answer = processor.decode(gen_part, skip_special_tokens=True)

            print(f"[{question_id}] {decoded_answer}")

            output_item = {
                "question_id": question_id,
                "category": category,
                "image_category": image_cat,
                "context": context,
                "question": question_text,
                "answer": decoded_answer
            }
            f_out.write(json.dumps(output_item, ensure_ascii=False) + "\n")

    print(f"All done! Results saved in {output_file}")

if __name__ == "__main__":
    main()


Loading model: google/gemma-3-4b-it


/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:4056: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/processing_auto.py:243: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:653: UserWarning: `do_sample` is set to `False`. However

[0] はい、その通りです！このイラストは宮崎駿監督のジブリ作品「君たちはどう生きるか」のワンシーンです。

特に、少年が羽を加工している様子や、全体的な作風から、宮崎駿監督の作品だと断定できます。

他に何か知りたいことはありますか？
[1] 映像に写っている少年は、鳥の羽を小型ナイフで加工しています。これは、書道や絵画などの芸術作品を作るための準備段階と考えられます。
[2] 画像に写っている鳥の羽の色は青色です。
[3] 机の上には2本の鉛筆が置かれています。
[4] はい、承知いたしました。画像について詳しく説明しますね。

このイラストは、宮崎駿監督の映画『君たちはどう生きるか』のワンシーンです。

**主な要素:**

*   **少年:** 画面中央に写っているのは、黒髪の少年です。彼は小型のナイフを使って、青色の鳥の羽を丁寧に加工しています。彼の集中した表情から、何か重要な作業をしていることがわかります。
*   **鳥の羽:** 机には2枚の青色の鳥の羽が置かれています。これは、映画の中で少年が鳥の羽を使って文字を書
[5] はい、承知いたしました。画像に写っているのは、宮崎駿監督のスタジオジブリ作品「千と千尋の神隠し」のワンシーンですね。

**作品のあらすじ**

この映画は、12歳の少女、千尋・荻野が、両親が不思議な力で神の世界に引き込まれてしまうことから始まる物語です。

**物語の主な流れ**

1.  **引っ越しと異変:** 千尋は、都会への引っ越しを余儀なくされます。新しい学校への不安と、両親が夜になると奇妙な行動をと
[6] はい、承知いたしました。画像の中で明らかに人間ではないのは、**カオナシ**です。

カオナシは、白い顔と特徴的な口ひげが特徴的な、ジブリ作品「千と千尋の神隠し」に登場する不思議な化け物です。

ご質問ありがとうございます。他に何かお手伝いできることはありますか？
[7] イラストの少女の名前は千尋です。
[8] はい、一番左にいるのは人間ではありません。カオナシという化け物です。

画像の説明では、カオナシは「大きな鼻と目、特徴的な髪型を持つ老婦人の湯婆婆」の隣にいると説明されています。カオナシは、千尋の物語の中では、人間ではない不思議な存在です。
[9] はい、承知いたしました。このイラストについて詳しく